# Table of Conetents

<a href='#isolation.py'>isolation.py</a>

<a href='#sample_players.py'>sample_players.py</a>

<a href='#game_agent.py'>game_agent.py</a>

<a href='#Testing'>Testing</a>



## isolation.py

In [66]:
"""
This file contains the `Board` class, which implements the rules for the
game Isolation as described in lecture, modified so that the players move
like knights in chess rather than queens.

You MAY use and modify this class, however ALL function signatures must
remain compatible with the defaults provided, and none of your changes will
be available to project reviewers.
"""
import random
import timeit
import time
from copy import copy

TIME_LIMIT_MILLIS = 150


class Board(object):
    """Implement a model for the game Isolation assuming each player moves like
    a knight in chess.

    Parameters
    ----------
    player_1 : object
        An object with a get_move() function. This is the only function
        directly called by the Board class for each player.

    player_2 : object
        An object with a get_move() function. This is the only function
        directly called by the Board class for each player.

    width : int (optional)
        The number of columns that the board should have.

    height : int (optional)
        The number of rows that the board should have.
    """
    BLANK = 0
    NOT_MOVED = None

    def __init__(self, player_1, player_2, width=7, height=7):
        self.width = width
        self.height = height
        self.move_count = 0
        self._player_1 = player_1
        self._player_2 = player_2
        self._active_player = player_1
        self._inactive_player = player_2

        # The last 3 entries of the board state includes initiative (0 for
        # player 1, 1 for player 2) player 2 last move, and player 1 last move
        self._board_state = [Board.BLANK] * (width * height + 3)
        self._board_state[-1] = Board.NOT_MOVED
        self._board_state[-2] = Board.NOT_MOVED

    def hash(self):
        return str(self._board_state).__hash__()

    @property
    def active_player(self):
        """The object registered as the player holding initiative in the
        current game state.
        """
        return self._active_player

    @property
    def inactive_player(self):
        """The object registered as the player in waiting for the current
        game state.
        """
        return self._inactive_player

    def get_opponent(self, player):
        """Return the opponent of the supplied player.

        Parameters
        ----------
        player : object
            An object registered as a player in the current game. Raises an
            error if the supplied object is not registered as a player in
            this game.

        Returns
        -------
        object
            The opponent of the input player object.
        """
        if player == self._active_player:
            return self._inactive_player
        elif player == self._inactive_player:
            return self._active_player
        raise RuntimeError("`player` must be an object registered as a player in the current game.")

    def copy(self):
        """ Return a deep copy of the current board. """
        new_board = Board(self._player_1, self._player_2, width=self.width, height=self.height)
        new_board.move_count = self.move_count
        new_board._active_player = self._active_player
        new_board._inactive_player = self._inactive_player
        new_board._board_state = copy(self._board_state)
        return new_board

    def forecast_move(self, move):
        """Return a deep copy of the current game with an input move applied to
        advance the game one ply.

        Parameters
        ----------
        move : (int, int)
            A coordinate pair (row, column) indicating the next position for
            the active player on the board.

        Returns
        -------
        isolation.Board
            A deep copy of the board with the input move applied.
        """
        new_board = self.copy()
        new_board.apply_move(move)
        return new_board

    def move_is_legal(self, move):
        """Test whether a move is legal in the current game state.

        Parameters
        ----------
        move : (int, int)
            A coordinate pair (row, column) indicating the next position for
            the active player on the board.

        Returns
        -------
        bool
            Returns True if the move is legal, False otherwise
        """
        idx = move[0] + move[1] * self.height
        return (0 <= move[0] < self.height and 0 <= move[1] < self.width and
                self._board_state[idx] == Board.BLANK)

    def get_blank_spaces(self):
        """Return a list of the locations that are still available on the board.
        """
        return [(i, j) for j in range(self.width) for i in range(self.height)
                if self._board_state[i + j * self.height] == Board.BLANK]

    def get_player_location(self, player):
        """Find the current location of the specified player on the board.

        Parameters
        ----------
        player : object
            An object registered as a player in the current game.

        Returns
        -------
        (int, int) or None
            The coordinate pair (row, column) of the input player, or None
            if the player has not moved.
        """
        if player == self._player_1:
            if self._board_state[-1] == Board.NOT_MOVED:
                return Board.NOT_MOVED
            idx = self._board_state[-1]
        elif player == self._player_2:
            if self._board_state[-2] == Board.NOT_MOVED:
                return Board.NOT_MOVED
            idx = self._board_state[-2]
        else:
            raise RuntimeError(
                "Invalid player in get_player_location: {}".format(player))
        w = idx // self.height
        h = idx % self.height
        return (h, w)

    def get_legal_moves(self, player=None):
        """Return the list of all legal moves for the specified player.

        Parameters
        ----------
        player : object (optional)
            An object registered as a player in the current game. If None,
            return the legal moves for the active player on the board.

        Returns
        -------
        list<(int, int)>
            The list of coordinate pairs (row, column) of all legal moves
            for the player constrained by the current game state.
        """
        if player is None:
            player = self.active_player
        return self.__get_moves(self.get_player_location(player))

    def apply_move(self, move):
        """Move the active player to a specified location.

        Parameters
        ----------
        move : (int, int)
            A coordinate pair (row, column) indicating the next position for
            the active player on the board.
        """
        idx = move[0] + move[1] * self.height
        last_move_idx = int(self.active_player == self._player_2) + 1
        self._board_state[-last_move_idx] = idx
        self._board_state[idx] = 1
        self._board_state[-3] ^= 1
        self._active_player, self._inactive_player = self._inactive_player, self._active_player
        self.move_count += 1

    def is_winner(self, player):
        """ Test whether the specified player has won the game. """
        return player == self._inactive_player and not self.get_legal_moves(self._active_player)

    def is_loser(self, player):
        """ Test whether the specified player has lost the game. """
        return player == self._active_player and not self.get_legal_moves(self._active_player)

    def utility(self, player):
        """Returns the utility of the current game state from the perspective
        of the specified player.

                    /  +infinity,   "player" wins
        utility =  |   -infinity,   "player" loses
                    \          0,    otherwise

        Parameters
        ----------
        player : object (optional)
            An object registered as a player in the current game. If None,
            return the utility for the active player on the board.

        Returns
        ----------
        float
            The utility value of the current game state for the specified
            player. The game has a utility of +inf if the player has won,
            a value of -inf if the player has lost, and a value of 0
            otherwise.
        """
        if not self.get_legal_moves(self._active_player):

            if player == self._inactive_player:
                return float("inf")

            if player == self._active_player:
                return float("-inf")

        return 0.

    def __get_moves(self, loc):
        """Generate the list of possible moves for an L-shaped motion (like a
        knight in chess).
        """
        if loc == Board.NOT_MOVED:
            return self.get_blank_spaces()

        r, c = loc
        directions = [(-2, -1), (-2, 1), (-1, -2), (-1, 2),
                      (1, -2), (1, 2), (2, -1), (2, 1)]
        valid_moves = [(r + dr, c + dc) for dr, dc in directions
                       if self.move_is_legal((r + dr, c + dc))]
        random.shuffle(valid_moves)
        return valid_moves

    def print_board(self):
        """DEPRECATED - use Board.to_string()"""
        return self.to_string()

    def to_string(self, symbols=['1', '2']):
        """Generate a string representation of the current game state, marking
        the location of each player and indicating which cells have been
        blocked, and which remain open.
        """
        p1_loc = self._board_state[-1]
        p2_loc = self._board_state[-2]

        col_margin = len(str(self.height - 1)) + 1
        prefix = "{:<" + "{}".format(col_margin) + "}"
        offset = " " * (col_margin + 3)
        out = offset + '   '.join(map(str, range(self.width))) + '\n\r'
        for i in range(self.height):
            out += prefix.format(i) + ' | '
            for j in range(self.width):
                idx = i + j * self.height
                if not self._board_state[idx]:
                    out += ' '
                elif p1_loc == idx:
                    out += symbols[0]
                elif p2_loc == idx:
                    out += symbols[1]
                else:
                    out += '-'
                out += ' | '
            out += '\n\r'

        return out

    def play(self, time_limit=TIME_LIMIT_MILLIS):
        """Execute a match between the players by alternately soliciting them
        to select a move and applying it in the game.

        Parameters
        ----------
        time_limit : numeric (optional)
            The maximum number of milliseconds to allow before timeout
            during each turn.

        Returns
        ----------
        (player, list<[(int, int),]>, str)
            Return multiple including the winning player, the complete game
            move history, and a string indicating the reason for losing
            (e.g., timeout or invalid move).
        """
        move_history = []

        time_millis = lambda: 1000 * timeit.default_timer()

        while True:

            legal_player_moves = self.get_legal_moves()
            game_copy = self.copy()

            move_start = time_millis()
            time_left = lambda : time_limit - (time_millis() - move_start)
            curr_move = self._active_player.get_move(game_copy, time_left)
            move_end = time_left()

            if curr_move is None:
                curr_move = Board.NOT_MOVED

            if move_end < 0:
                return self._inactive_player, move_history, "timeout"

            if curr_move not in legal_player_moves:
                if len(legal_player_moves) > 0:
                    return self._inactive_player, move_history, "forfeit"
                return self._inactive_player, move_history, "illegal move"

            move_history.append(list(curr_move))

            self.apply_move(curr_move)


## sample_players.py

In [45]:
"""This file contains a collection of player classes for comparison with your
own agent and example heuristic functions.

    ************************************************************************
    ***********  YOU DO NOT NEED TO MODIFY ANYTHING IN THIS FILE  **********
    ************************************************************************
"""

from random import randint


def null_score(game, player):
    """This heuristic presumes no knowledge for non-terminal states, and
    returns the same uninformative value for all other states.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state.
    """

    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    return 0.


def open_move_score(game, player):
    """The basic evaluation function described in lecture that outputs a score
    equal to the number of moves open for your computer player on the board.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    return float(len(game.get_legal_moves(player)))


def improved_score(game, player):
    """The "Improved" evaluation function discussed in lecture that outputs a
    score equal to the difference in the number of moves available to the
    two players.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - opp_moves)


def center_score(game, player):
    """Outputs a score equal to square of the distance from the center of the
    board to the position of the player.

    This heuristic is only used by the autograder for testing.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).

    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    w, h = game.width / 2., game.height / 2.
    y, x = game.get_player_location(player)
    return float((h - y)**2 + (w - x)**2)


class RandomPlayer():
    """Player that chooses a move randomly."""

    def get_move(self, game, time_left):
        """Randomly select a move from the available legal moves.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            A randomly selected legal move; may return (-1, -1) if there are
            no available legal moves.
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)
        return legal_moves[randint(0, len(legal_moves) - 1)]


class GreedyPlayer():
    """Player that chooses next move to maximize heuristic score. This is
    equivalent to a minimax search agent with a search depth of one.
    """

    def __init__(self, score_fn=open_move_score):
        self.score = score_fn

    def get_move(self, game, time_left):
        """Select the move from the available legal moves with the highest
        heuristic score.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            The move in the legal moves list with the highest heuristic score
            for the current game state; may return (-1, -1) if there are no
            legal moves.
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)
        _, move = max([(self.score(game.forecast_move(m), self), m) for m in legal_moves])
        return move


class HumanPlayer():
    """Player that chooses a move according to user's input."""

    def get_move(self, game, time_left):
        """
        Select a move from the available legal moves based on user input at the
        terminal.

        **********************************************************************
        NOTE: If testing with this player, remember to disable move timeout in
              the call to `Board.play()`.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            The move in the legal moves list selected by the user through the
            terminal prompt; automatically return (-1, -1) if there are no
            legal moves
        """
        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1, -1)

        print(game.to_string()) #display the board for the human player
        print(('\t'.join(['[%d] %s' % (i, str(move)) for i, move in enumerate(legal_moves)])))

        valid_choice = False
        while not valid_choice:
            try:
                index = int(input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')

            except ValueError:
                print('Invalid index! Try again.')

        return legal_moves[index]


if __name__ == "__main__":
    from isolation import Board

    # create an isolation board (by default 7x7)
    player1 = RandomPlayer()
    player2 = GreedyPlayer()
    game = Board(player1, player2)

    # place player 1 on the board at row 2, column 3, then place player 2 on
    # the board at row 0, column 5; display the resulting board state.  Note
    # that the .apply_move() method changes the calling object in-place.
    game.apply_move((2, 3))
    game.apply_move((0, 5))
    print(game.to_string())

    # players take turns moving on the board, so player1 should be next to move
    assert(player1 == game.active_player)

    # get a list of the legal moves available to the active player
    print(game.get_legal_moves())

    # get a successor of the current state by making a copy of the board and
    # applying a move. Notice that this does NOT change the calling object
    # (unlike .apply_move()).
    new_game = game.forecast_move((1, 1))
    assert(new_game.to_string() != game.to_string())
    print("\nOld state:\n{}".format(game.to_string()))
    print("\nNew state:\n{}".format(new_game.to_string()))

    # play the remainder of the game automatically -- outcome can be "illegal
    # move", "timeout", or "forfeit"
    winner, history, outcome = game.play()
    print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
    print(game.to_string())
    print("Move history:\n{!s}".format(history))


     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | 1 |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 

[(3, 5), (0, 2), (4, 4), (0, 4), (1, 1), (4, 2), (1, 5), (3, 1)]

Old state:
     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | 1 |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 


New state:
     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   | 1 |   |   |   |   |   | 
2  |   |   |   | - |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 


Winner: <__main__.RandomPlayer object at 0x110321c50>
Outcome: illegal move
     0   1   2   3   4  

## game_agent.py

In [70]:
"""Finish all TODO items in this file to complete the isolation project, then
test your agent's strength against a set of known agents using tournament.py
and include the results in your report.
"""


class SearchTimeout(Exception):
    """Subclass base exception for code clarity. """
    pass


def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    This should be the best heuristic function for your project submission.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - 2*opp_moves)


def custom_score_2(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    w, h = game.width / 2., game.height / 2.
    y, x = game.get_player_location(player)
    return -float((h - y)**2 + (w - x)**2)


def custom_score_3(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(-opp_moves)


class IsolationPlayer:
    """Base class for minimax and alphabeta agents -- this class is never
    constructed or tested directly.

    ********************  DO NOT MODIFY THIS CLASS  ********************

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    def __init__(self, search_depth=3, score_fn=custom_score, timeout=10.):
        self.search_depth = search_depth
        self.score = score_fn
        self.time_left = None
        self.TIMER_THRESHOLD = timeout


class MinimaxPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************

        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        if len(game.get_legal_moves()) > 0:
            best_move = game.get_legal_moves()[0]
        else:
            best_move = ()

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move

    def minimax(self, game, depth):
        """Implement depth-limited minimax search algorithm as described in
        the lectures.

        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        # TODO: finish this function!
        best_score = float("-inf")
        if len(game.get_legal_moves()) > 0:
            best_move = game.get_legal_moves()[0]
        else:
            best_move = ()

        v = float("-inf")
        for child_node in game.get_legal_moves():
            v = max(v, self.min_value(game.forecast_move(child_node), depth - 1))
            if v > best_score:
                best_score = v
                best_move = child_node
        return best_move


    def terminal_test(self, game):
        """ Return True if the game is over for the active player
        and False otherwise.
        """
        if game.get_legal_moves() == []:
            return True
        else:
            return False

    def max_value(self, game, depth):
        """ Return the value for a loss (-1) if the game is over,
        otherwise return the maximum value over all legal child
        nodes.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        if depth == 0 or self.terminal_test(game) == True:
            return self.score(game, self)
        else:
            v = float("-inf")
            for child_node in game.get_legal_moves():
                v = max(v, self.min_value(game.forecast_move(child_node), depth - 1))
            return v

    def min_value(self, game, depth):
        """ Return the value for a win (+1) if the game is over,
        otherwise return the minimum value over all legal child
        nodes.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        if depth == 0 or self.terminal_test(game) == True:
            return self.score(game, self)
        else:
            v = float("inf")
            for child_node in game.get_legal_moves():
                v = min(v, self.max_value(game.forecast_move(child_node), depth - 1))
            return v


class AlphaBetaPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using iterative deepening minimax
    search with alpha-beta pruning. You must finish and test this player to
    make sure it returns a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        Modify the get_move() method from the MinimaxPlayer class to implement
        iterative deepening search instead of fixed-depth search.

        **********************************************************************
        NOTE: If time_left() < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # TODO: finish this function!
        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        
        if len(game.get_legal_moves()) > 0:
            _, best_move = max([(self.score(game.forecast_move(m), self), m) for m in game.get_legal_moves()])
        else:
            best_move = ()

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            
            for depth in range(0,10000000000000000000000):
                best_move = self.alphabeta(game, depth=depth)
            return best_move

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move

    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf")):
        """Implement depth-limited minimax search with alpha-beta pruning as
        described in the lectures.

        This should be a modified version of ALPHA-BETA-SEARCH in the AIMA text
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Alpha-Beta-Search.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        # TODO: finish this function!
        best_score = float("-inf")
        if len(game.get_legal_moves()) > 0:
            _, best_move = max([(self.score(game.forecast_move(m), self), m) for m in game.get_legal_moves()])
        else:
            best_move = ()

        v = float("-inf")
        for child_node in game.get_legal_moves():
            v = max(v, self.min_value(game.forecast_move(child_node), depth - 1, alpha, beta))   
            if v > best_score:
                best_score = v
                best_move = child_node
            alpha = max(alpha, v)
            if alpha >= beta:
                break
        return best_move


    def terminal_test(self, game):
        """ Return True if the game is over for the active player
        and False otherwise.
        """
        if game.get_legal_moves() == []:
            return True
        else:
            return False

    def max_value(self, game, depth, alpha, beta):
        """ Return the value for a loss (-1) if the game is over,
        otherwise return the maximum value over all legal child
        nodes.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        if depth == 0 or self.terminal_test(game) == True:
            return self.score(game, self)
        else:
            v = float("-inf")
            for child_node in game.get_legal_moves():
                v = max(v, self.min_value(game.forecast_move(child_node), depth - 1, alpha, beta))
                alpha = max(alpha, v)
                if alpha >= beta:
                    break
            return v

    def min_value(self, game, depth, alpha, beta):
        """ Return the value for a win (+1) if the game is over,
        otherwise return the minimum value over all legal child
        nodes.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        if depth == 0 or self.terminal_test(game) == True:
            return self.score(game, self)
        else:
            v = float("inf")
            for child_node in game.get_legal_moves():
                v = min(v, self.max_value(game.forecast_move(child_node), depth - 1, alpha, beta))
                beta = min(beta, v)
                if alpha >= beta:
                    break
            return v
    

## Testing

### Single Play

In [48]:
player1 = MinimaxPlayer()
player2 = RandomPlayer()
game = Board(player1, player2)

# # place player 1 on the board at row 2, column 3, then place player 2 on
# # the board at row 0, column 5; display the resulting board state.  Note
# # that the .apply_move() method changes the calling object in-place.
# game.apply_move((2, 3))
# game.apply_move((0, 5))
# print(game.to_string())

# # players take turns moving on the board, so player1 should be next to move
# assert(player1 == game.active_player)

# # get a list of the legal moves available to the active player
# print(game.get_legal_moves())

# # get a successor of the current state by making a copy of the board and
# # applying a move. Notice that this does NOT change the calling object
# # (unlike .apply_move()).
# new_game = game.forecast_move((1, 1))
# assert(new_game.to_string() != game.to_string())
# print("\nOld state:\n{}".format(game.to_string()))
# print("\nNew state:\n{}".format(new_game.to_string()))

# play the remainder of the game automatically -- outcome can be "illegal
# move", "timeout", or "forfeit"
winner, history, outcome = game.play()
print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
print(game.to_string())
print("Move history:\n{!s}".format(history))


Winner: <__main__.MinimaxPlayer object at 0x110334cc0>
Outcome: illegal move
     0   1   2   3   4   5   6
0  | - | - |   | 2 | - | - |   | 
1  |   | - | - | - | - | - | - | 
2  | - | - | - | - | - |   | - | 
3  | 1 | - | - | - | - | - | - | 
4  |   | - | - | - | - | - | - | 
5  |   | - |   | - | - |   | - | 
6  |   |   |   | - |   |   |   | 

Move history:
[[0, 0], [5, 4], [2, 1], [4, 6], [4, 2], [3, 4], [6, 3], [1, 3], [5, 1], [0, 5], [4, 3], [2, 6], [3, 1], [4, 5], [1, 2], [2, 4], [2, 0], [3, 6], [0, 1], [4, 4], [2, 2], [5, 6], [1, 4], [3, 5], [3, 3], [1, 6], [4, 1], [0, 4], [5, 3], [2, 3], [3, 2], [1, 5], [1, 1], [0, 3], [3, 0]]


In [22]:
game.get_legal_moves()

[]

### Single Play

In [62]:
# player1 = GreedyPlayer(score_fn=improved_score)
player1 = RandomPlayer()
player2 = AlphaBetaPlayer(score_fn=custom_score_2)
game = Board(player1, player2)

winner, history, outcome = game.play()
print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
print(game.to_string())
print("Move history:\n{!s}".format(history))


Winner: <__main__.AlphaBetaPlayer object at 0x110497390>
Outcome: illegal move
     0   1   2   3   4   5   6
0  | 2 |   | - |   |   |   |   | 
1  | - |   |   |   |   |   |   | 
2  |   | - | - | - | - |   |   | 
3  | - | - | - | - | - | - | - | 
4  |   | - | - | - | - | - |   | 
5  | 1 | - | - | - | - | - | - | 
6  | - |   | - | - | - |   |   | 

Move history:
[[3, 0], [4, 4], [5, 1], [3, 2], [6, 3], [5, 3], [5, 5], [3, 4], [3, 6], [4, 2], [2, 4], [5, 4], [4, 3], [3, 3], [2, 2], [4, 5], [1, 0], [6, 4], [3, 1], [5, 6], [5, 2], [3, 5], [6, 0], [2, 3], [4, 1], [0, 2], [6, 2], [2, 1], [5, 0], [0, 0]]


### Multiple Games - Minimax vs. AlphaBeta (score_fn = own_moves - 2*opp_moves)

In [11]:
def find_between( s, first, last ):
    start = s.index( first ) + len( first )
    end = s.index( last, start )
    return s[start:end]

player1 = MinimaxPlayer()
player2 = AlphaBetaPlayer(score_fn=custom_score_2)

cnt_minimax = 0
cnt_alphabeta = 0
cnt_forfeit = 0
cnt_illegal = 0

for i in range(10):
    game = Board(player1, player2)
    winner, history, outcome = game.play()
    winner = find_between(str(winner),'.', ' ')
    if winner == 'AlphaBetaPlayer':
        cnt_alphabeta += 1
    else:
        cnt_minimax += 1
    if outcome == 'forfeit':
        cnt_forfeit += 1
    elif outcome == 'illegal move':
        cnt_illegal += 1
    
print('AlphaBeta Won: ' + str(cnt_alphabeta) + ' times')
print('Minimax Won: ' + str(cnt_minimax) + ' times')
# print('Number of forfeited games: ' + str(cnt_forfeit))
# print('Number of completed games: ' + str(cnt_illegal))
    

AlphaBeta Won: 6 times
Minimax Won: 4 times


### Multiple Games - Minimax vs. AlphaBeta (score_fn = own_moves - 3*opp_moves)

In [ ]:
def find_between( s, first, last ):
    start = s.index( first ) + len( first )
    end = s.index( last, start )
    return s[start:end]

player1 = MinimaxPlayer()
player2 = AlphaBetaPlayer(score_fn=custom_score)

cnt_minimax = 0
cnt_alphabeta = 0
cnt_forfeit = 0
cnt_illegal = 0

for i in range(100):
    game = Board(player1, player2)
    winner, history, outcome = game.play()
    winner = find_between(str(winner),'.', ' ')
    if winner == 'AlphaBetaPlayer':
        cnt_alphabeta += 1
    else:
        cnt_minimax += 1
    if outcome == 'forfeit':
        cnt_forfeit += 1
    elif outcome == 'illegal move':
        cnt_illegal += 1
    
print('AlphaBeta Won: ' + str(cnt_alphabeta) + ' times')
print('Minimax Won: ' + str(cnt_minimax) + ' times')
# print('Number of forfeited games: ' + str(cnt_forfeit))
# print('Number of completed games: ' + str(cnt_illegal))
    

### Multiple Games - Greedy vs. AlphaBeta (score_fn = own_moves - 2*opp_moves)

In [67]:
start_time = time.time()

def find_between( s, first, last ):
    start = s.index( first ) + len( first )
    end = s.index( last, start )
    return s[start:end]

player1 = GreedyPlayer(score_fn=improved_score)
player2 = AlphaBetaPlayer(score_fn=custom_score)

cnt_greedy = 0
cnt_alphabeta = 0
cnt_forfeit = 0
cnt_illegal = 0

for i in range(100):
    game = Board(player1, player2)
    winner, history, outcome = game.play()
    winner = find_between(str(winner),'.', ' ')
    if winner == 'AlphaBetaPlayer':
        cnt_alphabeta += 1
    else:
        cnt_greedy += 1
    if outcome == 'forfeit':
        cnt_forfeit += 1
    elif outcome == 'illegal move':
        cnt_illegal += 1
    
print('AlphaBeta Won: ' + str(cnt_alphabeta) + ' times')
print('Greedy Won: ' + str(cnt_greedy) + ' times')
# print('Number of forfeited games: ' + str(cnt_forfeit))
# print('Number of completed games: ' + str(cnt_illegal))


elapsed_time = time.time() - start_time
print(elapsed_time)

AlphaBeta Won: 99 times
Greedy Won: 1 times
189.66492009162903


### Multiple Games - Greedy vs. AlphaBeta (score_fn = prefer centre)

In [15]:
start_time = time.time()

def find_between( s, first, last ):
    start = s.index( first ) + len( first )
    end = s.index( last, start )
    return s[start:end]

player1 = GreedyPlayer(score_fn=improved_score)
player2 = AlphaBetaPlayer(score_fn=custom_score_2)

cnt_greedy = 0
cnt_alphabeta = 0
cnt_forfeit = 0
cnt_illegal = 0

for i in range(100):
    game = Board(player1, player2)
    winner, history, outcome = game.play()
    winner = find_between(str(winner),'.', ' ')
    if winner == 'AlphaBetaPlayer':
        cnt_alphabeta += 1
    else:
        cnt_greedy += 1
    if outcome == 'forfeit':
        cnt_forfeit += 1
    elif outcome == 'illegal move':
        cnt_illegal += 1
    
print('AlphaBeta Won: ' + str(cnt_alphabeta) + ' times')
print('Greedy Won: ' + str(cnt_greedy) + ' times')
# print('Number of forfeited games: ' + str(cnt_forfeit))
# print('Number of completed games: ' + str(cnt_illegal))
    
    
elapsed_time = time.time() - start_time
print(elapsed_time)

AlphaBeta Won: 0 times
Greedy Won: 100 times


### Multiple Games - Greedy vs. AlphaBeta (score_fn = prefer corner)

In [16]:
start_time = time.time()

def find_between( s, first, last ):
    start = s.index( first ) + len( first )
    end = s.index( last, start )
    return s[start:end]

player1 = GreedyPlayer(score_fn=improved_score)
player2 = AlphaBetaPlayer(score_fn=custom_score_3)

cnt_greedy = 0
cnt_alphabeta = 0
cnt_forfeit = 0
cnt_illegal = 0

for i in range(100):
    game = Board(player1, player2)
    winner, history, outcome = game.play()
    winner = find_between(str(winner),'.', ' ')
    if winner == 'AlphaBetaPlayer':
        cnt_alphabeta += 1
    else:
        cnt_greedy += 1
    if outcome == 'forfeit':
        cnt_forfeit += 1
    elif outcome == 'illegal move':
        cnt_illegal += 1
    
print('AlphaBeta Won: ' + str(cnt_alphabeta) + ' times')
print('Greedy Won: ' + str(cnt_greedy) + ' times')
# print('Number of forfeited games: ' + str(cnt_forfeit))
# print('Number of completed games: ' + str(cnt_illegal))


elapsed_time = time.time() - start_time
print(elapsed_time)

AlphaBeta Won: 0 times
Greedy Won: 100 times
